In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

SUMMARY_DIR = 'log_9_3'
BATCH_SIZE = 100
TRAIN_STEPS = 30000

def variable_summaries(var):
    with tf.name_scope('summaries'):
        tf.summary.histogram('histogram', var)
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
    
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
            variable_summaries(weights)
        
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.0, shape=[output_dim]))
            variable_summaries(biases)
            
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('histogram', preactivate)
            
        activations = act(preactivate, name='activation')
        tf.summary.histogram('histogram', activations)
        
        return activations
    
def main():
    if os.path.exists("/Users"):
        mnist = input_data.read_data_sets("/Users/colinzuo/work/github/personal_study/"
                                          "tensorflow/book_caicloud/mnist/", one_hot=True)
    else:
        mnist = input_data.read_data_sets("D:\\work\\DataAnalysis\\study\\mnist", one_hot=True)
        
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
        
    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', image_shaped_input, 10)
        
    hidden1 = nn_layer(x, 784, 500, 'layer1')
    y = nn_layer(hidden1, 500, 10, 'layer2', act=tf.identity)
    
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
        tf.summary.scalar('cross_entropy', cross_entropy)
        
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
        
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        with tf.name_scope('accuray'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
        tf.global_variables_initializer().run()
        
        for i in range(TRAIN_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            summary, _ = sess.run([merged, train_step], feed_dict={x: xs, y_: ys})
            summary_writer.add_summary(summary, i)
            
        summary_writer.close()
        
main()

Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-labels-idx1-ubyte.gz
